In [16]:
import numpy as np
import random
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
# Memuat file Excel
def load_matrix(file_path):
    df = pd.read_excel(file_path, index_col=0)
    df.columns = df.columns.astype(str).str.strip()
    df.index = df.index.astype(str).str.strip()
    return df

In [3]:
# Load DataFrame (asumsi menggunakan fungsi `load_matrix()`)
file_path = "matrix.xlsx"
markov_matrix = load_matrix(file_path)

# Pastikan tipe data DataFrame adalah float
markov_matrix = markov_matrix.astype(float)

epsilon = 1e-6  # Nilai kecil untuk normalisasi

# Perbaiki state yang tidak memiliki transisi
for index, row in markov_matrix.iterrows():
    if np.isclose(np.sum(row), 1.0):  # State absorbing tetap dipertahankan
        continue
    elif np.isclose(np.sum(row), 0.0):  # Jika tidak ada transisi, tambahkan probabilitas kecil
        markov_matrix.loc[index] = np.ones(len(row), dtype=np.float64) * epsilon
        markov_matrix.loc[index, index] = 1 - (epsilon * (len(row) - 1))  # Menjaga normalisasi

# Pastikan matriks transisi tetap valid (jumlah setiap baris harus 1)
markov_matrix = markov_matrix.div(markov_matrix.sum(axis=1), axis=0)

# Konversi DataFrame ke NumPy sebelum matrix power
markov_numpy = markov_matrix.to_numpy()

print("Baris:", len(markov_matrix), "Kolom:", len(markov_matrix.columns))

Baris: 2800 Kolom: 2800


In [ ]:
# Nama baris yang dicari
# kurangi_depart = "Worker_Yard Operations Department_Gate Operator"
# tambah_depart= "Worker_Yard Operations Department_Tallyman Loading Discharge"


depart_kurang = {"Worker_Yard Operations Department_Gate Operator" : 4, "Worker_Yard Operations Department_Forklift Operator Jakarta Branch": 2}
# quantity_kurang = [4,2]
depart_tambah = {"Worker_Yard Operations Department_Tallyman Loading Discharge":2, "Worker_Yard Operations Department_Gate Operator":2,"Worker_Ship Operations Department_Shore Crane Operator":2}
# quantity_tambah = [2,2,2]

# nanti input dan output berupa 2 array dictionary yang berisi value nama seperti Worker_Yard Operations Department_Gate Operator dan quantity (integer lebih dari 1)

if all(k in markov_matrix.index for k in depart_kurang) & all(t in markov_matrix.index for t in depart_tambah):
    output = []
    for index1, q1 in depart_kurang.items():
        new_posisi = []
        new_probabilty = []
        search_array = markov_matrix.loc[index1].to_numpy()
        mc_p1 = np.linalg.matrix_power(markov_numpy,1)
        hasil = np.dot(search_array, mc_p1)
        for index2, q2 in depart_tambah.items():
            new_posisi.append((markov_matrix.columns.get_loc(index2)))
            temp = hasil[(markov_matrix.columns.get_loc(index2))]
            new_probabilty.append((temp))
            print(f"Probabilitas menuju state '{index2}': {temp}")            
        print("--------------------------------------------------------------------")
else:
    print("error")

Probabilitas menuju state 'Worker_Yard Operations Department_Tallyman Loading Discharge': 0.17551023265306126
Probabilitas menuju state 'Worker_Yard Operations Department_Gate Operator': 0.29641198870431895
Probabilitas menuju state 'Worker_Ship Operations Department_Shore Crane Operator': 0.005714314285714285
4
---------------------------------------------------------------------
Probabilitas menuju state 'Worker_Yard Operations Department_Tallyman Loading Discharge': 2.49999999999998e-07
Probabilitas menuju state 'Worker_Yard Operations Department_Gate Operator': 2.49999999999998e-07
Probabilitas menuju state 'Worker_Ship Operations Department_Shore Crane Operator': 2.49999999999998e-07
2
---------------------------------------------------------------------


In [40]:

# Misal markov_matrix dan markov_numpy sudah didefinisikan sebelumnya
# Contoh dictionary input:
depart_kurang = {
    "Worker_Yard Operations Department_Gate Operator": 4, 
    "Worker_Yard Operations Department_Forklift Operator Jakarta Branch": 2
}
depart_tambah = {
    "Worker_Yard Operations Department_Tallyman Loading Discharge": 2, 
    "Worker_Yard Operations Department_Gate Operator": 2,
    "Worker_Ship Operations Department_Shore Crane Operator": 2
}

# Pastikan nama row/column ada di markov_matrix
if all(k in markov_matrix.index for k in depart_kurang) and all(t in markov_matrix.index for t in depart_tambah):
    
    output = []  # Output akhir: list yang berisi list alokasi untuk tiap depart_kurang
    # Buat salinan kapasitas global untuk depart_tambah (akan dikurangi seiring alokasi)
    remaining_capacity = depart_tambah.copy()
    # Urutan key dari depart_tambah (penting agar output konsisten)
    depart_tambah_keys = list(depart_tambah.keys())
    
    # Loop tiap depart_kurang
    for index1, q1 in depart_kurang.items():
        new_probability = []
        
        # Ambil baris dari markov_matrix untuk depart_kurang
        search_array = markov_matrix.loc[index1].to_numpy()
        mc_p1 = np.linalg.matrix_power(markov_numpy, 1)
        hasil = np.dot(search_array, mc_p1)
        
        # Hitung probabilitas untuk tiap depart_tambah (sesuai urutan depart_tambah_keys)
        for key in depart_tambah_keys:
            col_index = markov_matrix.columns.get_loc(key)
            prob = hasil[col_index]
            new_probability.append(prob)
            print(f"Probabilitas menuju state '{key}': {prob}")
        print("---------------------------------------------------------------------")
        
        # Normalisasi probabilitas agar jumlahnya 1 (jika total > 0)
        total_prob = sum(new_probability)
        if total_prob > 0:
            norm_probability = [p / total_prob for p in new_probability]
        else:
            norm_probability = [1 / len(new_probability)] * len(new_probability)
        
        # Inisialisasi alokasi untuk baris ini, dalam bentuk list dengan panjang sama dengan jumlah depart_tambah
        allocation = [0] * len(depart_tambah_keys)
        items_to_allocate = q1
        
        # Lakukan alokasi item sebanyak q1 unit
        while items_to_allocate > 0:
            # Buat daftar kandidat (indeks) dari depart_tambah yang masih memiliki sisa kapasitas
            candidates = []
            candidate_weights = []
            for i, key in enumerate(depart_tambah_keys):
                if remaining_capacity[key] > 0:
                    candidates.append(i)
                    candidate_weights.append(norm_probability[i])
            
            # Jika tidak ada kandidat tersisa, keluarkan pesan peringatan dan hentikan alokasi untuk baris ini
            if not candidates:
                print("Tidak ada kapasitas tersisa untuk alokasi!")
                break
            
            # Pilih secara acak salah satu kandidat menggunakan bobot
            chosen_idx = random.choices(candidates, weights=candidate_weights, k=1)[0]
            allocation[chosen_idx] += 1
            remaining_capacity[depart_tambah_keys[chosen_idx]] -= 1
            items_to_allocate -= 1
        
        output.append(allocation)
    
    print("Distribusi akhir:")
    print(output)
else:
    print("error")


Probabilitas menuju state 'Worker_Yard Operations Department_Tallyman Loading Discharge': 0.17551023265306126
Probabilitas menuju state 'Worker_Yard Operations Department_Gate Operator': 0.29641198870431895
Probabilitas menuju state 'Worker_Ship Operations Department_Shore Crane Operator': 0.005714314285714285
---------------------------------------------------------------------
Probabilitas menuju state 'Worker_Yard Operations Department_Tallyman Loading Discharge': 2.49999999999998e-07
Probabilitas menuju state 'Worker_Yard Operations Department_Gate Operator': 2.49999999999998e-07
Probabilitas menuju state 'Worker_Ship Operations Department_Shore Crane Operator': 2.49999999999998e-07
---------------------------------------------------------------------
Distribusi akhir:
[[1, 2, 1], [1, 0, 1]]


level = ['Director', 'Expatriat', 'General Manager', 'Junior Manager', 'Middle Manager', 'Non Grade', 'President Director', 'Senior Manager', 'Senior Staff', 'Staff', 'Team Leader', 'Trainee', 'Worker']
department = ['Account Receivable Core Department', 'Account Receivable Department', 'Accounting Department', 'Administrasi Depo', 'Administration Department', 'BOD', 'Branch Class B East', 'Branch Class C East', 'Branches Management Department', 'Bulk Management Department', 'Bunker And Overseas Purchasing Department', 'Business Development Department', 'Car Carrier', 'Chinese Desk', 'Commercial Department', 'Commercial Jakarta Department', 'Continuous Improvement Department', 'Core Department', 'Corporate Account Department', 'Corporate Communication', 'Customer Service Department', 'DFF', 'DPA Department', 'DPL', 'Facility Electrical Department', 'Feeder, SOC & International Trade Sales Department', 'Finance & Administration Department', 'Finance Department', 'Finance and Accounting 

In [ ]:
import pymysql

In [ ]:
PATH = "FRONT_END -> markov.py <-> database -> classification -> random_mutation_people -> FRONT_END" 

In [ ]:
level = ['Director', 'Expatriat', 'General Manager', 'Junior Manager', 'Middle Manager', 'Non Grade', 'President Director', 'Senior Manager', 'Senior Staff', 'Staff', 'Team Leader', 'Trainee', 'Worker']
department = ['Account Receivable Core Department', 'Account Receivable Department', 'Accounting Department', 'Administrasi Depo', 'Administration Department', 'BOD', 'Branch Class B East', 'Branch Class C East', 'Branches Management Department', 'Bulk Management Department', 'Bunker And Overseas Purchasing Department', 'Business Development Department', 'Car Carrier', 'Chinese Desk', 'Commercial Department', 'Commercial Jakarta Department', 'Continuous Improvement Department', 'Core Department', 'Corporate Account Department', 'Corporate Communication', 'Customer Service Department', 'DFF', 'DPA Department', 'DPL', 'Facility Electrical Department', 'Feeder, SOC & International Trade Sales Department', 'Finance & Administration Department', 'Finance Department', 'Finance and Accounting Department', 'Fleet Department', 'Fleet Jakarta Department', 'General Accounting Department', 'General Affairs Department', 'Global Freight & Logistics', 'HR System and Information Department', 'HRBP Department', 'Health Safety Environment Department', 'Heavy Equipment Department', 'Human Capital', 'Human Capital & Corporate Affairs', 'Human Capital & Corporate Affairs Jakarta Department', 'Human Capital Department', 'Human Resources Change Agent', 'Human Resources Change Agent Department', 'Human Resources Development Department', 'Human Resources Development NPTI', 'Human Resources Information System', 'Human Resources Information System Department', 'Human Resources Service Excellence Department', 'Human Resources System Department', 'Human Resources and General Affair Department', 'IT Application Development Department', 'IT DevOps Department', 'IT Development Department', 'IT Network & Infrastructure Department', 'IT Planning & Business Process Department', 'IT Software Development & Business Process Analyst Department', 'Industrial Relation Department', 'Information Technology & Technical Support Dept', 'Information Technology System Department', 'Inland Service Department', 'Insurance & Claim Department', 'Internal Audit Department', 'Learning And Development Department', 'Legal & Claim Department', 'Legal Department', 'Logistic Department', 'Machinery Department', 'Management Accounting Department', 'Management Trainee', 'Management Trainee Department', 'Marketing & Communication Department', 'Marketing Department', 'Marketing Department (Domestic Trade)', 'Marketing Department (International Trade)', 'Marketing Inbound & SOC Department', 'Medical Department', 'National Sales Department', 'Nautical Department', 'Office of Corporate Communication', 'Office of Strategy Management Department', 'Operation', 'Operation Jakarta Department', 'Operations & Administration Core Department', 'Operations Department', 'Operations Jakarta Department', 'Operations NPTI', 'Organization Development Department', 'Organizational & Talent Management', 'Outports Department', 'Overseas Purchasing Department', 'Performance Excellence Department', 'Personnel And Industrial Relation Department', 'Personnel Branches Department', 'Personnel Surabaya Department', 'Process Excellence Department', 'Procurement Department', 'Production Department', 'Project and Building Maintenance Department', 'Public Affair, Industrial & Employee Relation Department', 'Public Affairs & Relation Department', 'Public Affairs And Security Department', 'Public Affairs Department', 'Purchasing Departement', 'Quality Assurance & Continuous Improvement', 'Quality Assurance Department', 'Quality Control Departement', 'Quality Health Safety And Enviroment Department', 'R', 'Recruitment Department', 'Regional East Department', 'Regional West Department', 'Rendal Department', 'Sales Department', 'Sales Inbound & SOC Department', 'Salvage Department', 'Sekretariat', 'Services Department', 'Services Department (Domestic Trade)', 'Services Department (International Trade)', 'Ship Operation Department (N)', 'Ship Operations Department', 'Ship Personnel Management Department', 'Special Project', 'Special Project & Corporate Representative', 'Strategy Development And Implementation Department', 'Surabaya Branch Office Department', 'TPIL', 'Talent Management & OD Department', 'Talent Management Department', 'Technical Department', 'Terminal Operator Surabaya', 'Trade Department', 'Treasury Department', 'Vendor Management', 'Workshop Department', 'Yard Development Department', 'Yard Operations Department', 'Yield Excellent Service Department']
job = ['A/P Cashier', 'A/P Commercial & Operations Coordinator', 'A/P Commercial & Operations Supervisor', 'A/P Commercial and Operations Staff', 'A/P Coordinator', 'A/P Core Assistant Manager', 'A/P Core Coordinator', 'A/P Core Logistic Assistant Manager', 'A/P Core Logistic Staff', 'A/P Core Staff', 'A/P Core Supervisor', 'A/P Fleet Coordinator', 'A/P Fleet Staff', 'A/P Fleet Supervisor', 'A/P Head Office Staff', 'A/P Head Office Supervisor', 'A/P Human Capital & Corporate Affairs Staff', 'A/P Human Capital & Corporate Affairs Supervisor', 'A/P Senior Staff (JKT)', 'A/P Staff', 'A/P Staff (Jakarta)', 'A/P Supervisor', 'A/R & Document Assistant Manager', 'A/R Accounting Staff', 'A/R Accounting Supervisor', 'A/R Administration Staff', 'A/R Affiliated Companies Supervisor', 'A/R Assistant Manager', 'A/R Cashier', 'A/R Collector', 'A/R Core & Logistic Assistant Manager', 'A/R Core Manager', 'A/R Core Staff', 'A/R Core Supervisor', 'A/R Customer Care Staff', 'A/R Finance Administration', 'A/R Finance And Administration Staff Branch Class A West', 'A/R Finance And Administration Staff Branch Class B West', 'A/R Finance And Administration Staff Jakarta Branch', 'A/R Finance Assistant Manager', 'A/R Finance Staff', 'A/R Finance Supervisor', 'A/R Manager', 'A/R Staff', 'A/R Staff (Jakarta)', 'A/R Supervisor', 'Acc. Management & Acc. Dev. Cust. Service Executive (Jakarta)', 'Acc. Management & Acc. Dev. Cust. Service Executive (Surabaya)', 'Account Assistant Supervisor (AM-SBY)', 'Account Development Manager (Jakarta)', 'Account Development Manager (Surabaya)', 'Account Executive', 'Account Executive (AD-JKT)', 'Account Executive (AD-SBY)', 'Account Executive (AM-JKT)', 'Account Executive (AM-SBY)', 'Account Management Manager (Jakarta)', 'Account Management Manager (Surabaya)', 'Account Manager', 'Account Manager (AD-JKT)', 'Account Manager (AD-SBY)', 'Account Manager (AM-JKT)', 'Account Manager (AM-SBY)', 'Account Manager Branch Class A East .', 'Account Manager Branch Class B East', 'Account Manager Branch Class C East', 'Account Manager Branch Class C West', 'Account Officer (AD-JKT)', 'Account Officer (AM-JKT)', 'Account Officer (AM-SBY)', 'Account Payable Assistant Manager', 'Account Receivable Core Assistant Manager', 'Accounting Assistant Manager', 'Accounting Assistant Supervisor', 'Accounting GL Staff', 'Accounting Manager', 'Accounting Staff', 'Accounting Supervisor', 'Accounts Payable Staff', 'Accounts Receivable Staff', 'Admin Staff', 'Administration Assistant Manager', 'Administration Container Inventory Control', 'Administration Ship Operation', 'Administration Support', 'Administration Support (Jakarta)', 'Administration Workshop Staff', 'Administration Yard Staff', 'Advisor', 'Advisor Jakarta', 'Advisor Staff', 'Affiliated Companies Accounting Assistant Manager', 'Affiliated Companies Accounting Staff', 'Air Conditioner & Refrigerator Electrician', 'Analyst DPL', 'Area Driver', 'Area Driver Branch Class A West', 'Area Driver Branch Class B West', 'Area Office Boy', 'Area Office Girl', 'Area Phone Operator', 'Artificial Intelligence Project Consolidator', 'Assessment Staff', 'Asset & Inventory Staff', 'Asset & Inventory Supervisor', 'Asset Inventory and Repair Supervisor', 'Assistant Cook', 'Assistant GM Human Capital', 'Assistant GM Human Capital & Corporate Affairs', 'Assistant GM Human Capital and Corporate Affairs', 'Assistant HOD Pipping', 'Assistant HOD Shipwright & Dock Master', 'Assistant Head of Corporate Communications', 'Assistant Manager Commercial Technical', 'Assistant Manager Finance', 'Assistant Manager Human Resources & General Affair', 'Assistant Manager IT Development', 'Assistant Manager Marketing', 'Assistant Manager Marketing (Domestic Trade)', 'Assistant Manager Marketing (International Trade)', 'Assistant Manager Marketing Inbound', 'Assistant Manager Outports', 'Assistant Manager QC', 'Assistant Manager Services', 'Assistant Manager Services (Domestic Trade)', 'Assistant Manager Workshop', 'Assistant Network Administrator', 'Assistant Operation Head Ambon', 'Assistant Operation Head Batam', 'Assistant Operation Head Berau', 'Assistant Operation Head Bitung', 'Assistant Operation Head Branch Class A East', 'Assistant Operation Head Branch Class A West', 'Assistant Operation Head Branch Class B East', 'Assistant Operation Head Branch Class B West', 'Assistant Operation Head Branch Class C East', 'Assistant Operation Head Branch Class C West', 'Assistant Operation Head Branch Class D East', 'Assistant Operation Head Branch Class D West', 'Assistant Operation Head Branch Class E West', 'Assistant Operation Head Jayapura', 'Assistant Operation Head Manokwari', 'Assistant Operation Head Medan', 'Assistant Operation Head Merauke', 'Assistant Operation Head Sorong', 'Assistant Painting', 'Assistant Pilot', 'Assistant Pilot Assistant Supervisor', 'Assistant Pilot Staff', 'Assistant Pilot Supervisor', 'Assistant Port Captain', 'Assistant Quality Control', 'Assistant Secretary', 'Assistant Superintendent', 'Assistant Superintendent Jakarta Branch', 'Assistant Supervisor HSE', 'Assistant Supervisor Hull', 'Assistant Supervisor Internal Audit', 'Assistant Supervisor Painting', 'Assistant Supervisor Programmer', 'Assistant System Administrator', 'Asst Manager Production', 'Asst. Purchasing Adm.', 'BA Core Staff', 'BA Core Supervisor', 'Back End & Corp. Account Cust. Service Manager', 'Back End Customer Service Executive (Jakarta)', 'Back End Customer Service Executive (Surabaya)', 'Back End Customer Service Team Leader (Jakarta)', 'Back End Customer Service Team Leader (Surabaya)', 'Branch & Call Center Customer Service Manager', 'Branch & Call Center Customer Service Manager Project', 'Branch Account Management', 'Branch Account Management Officer', 'Branch Administration Staff', 'Branch Administration Staff Ambon', 'Branch Administration Staff Branch Class A East', 'Branch Administration Staff Branch Class A West', 'Branch Administration Staff Branch Class B East', 'Branch Administration Staff Branch Class B West', 'Branch Administration Staff Branch Class C East', 'Branch Administration Staff Branch Class C West', 'Branch Administration Staff Branch Class D East', 'Branch Administration Supervisor', 'Branch Administration Supervisor Branch Class A East', 'Branch Administration Supervisor Branch Class A West', 'Branch Administration Supervisor Branch Class B East', 'Branch Administration Supervisor Branch Class B West', 'Branch Administration Supervisor Branch Class C East', 'Branch Administration Supervisor Branch Class C West', 'Branch Administration Supervisor Branch Class D West', 'Branch Administrator Branch Class A West', 'Branch Administrator Branch Class B East', 'Branch Administrator Branch Class B West', 'Branch Administrator Branch Class C East', 'Branch Administrator Branch Class C West', 'Branch Finance Coordinator', 'Branch Finance Staff', 'Branch Finance Supervisor', 'Branch Manager', 'Branch Manager Batulicin', 'Branch Manager Biak', 'Branch Manager Branch Class A East', 'Branch Manager Branch Class A West', 'Branch Manager Branch Class B East', 'Branch Manager Branch Class B West', 'Branch Manager Branch Class C East', 'Branch Manager Branch Class C West', 'Branch Manager Branch Class D East', 'Branch Manager Branch Class D West', 'Branch Manager Branch Class E East', 'Branch Manager Branch Class E West', 'Branch Manager Class C West (Palembang)', 'Branch Manager Fak-fak', 'Branch Manager Gorontalo', 'Branch Manager Kaimana', 'Branch Manager Nabire', 'Branch Manager Nunukan', 'Branch Manager Palu', 'Branch Manager Serui', 'Branch Manager TPIL Surabaya', 'Branch Manager Timika', 'Branch Manager Tual', 'Branch Operation Administration Staff West', 'Branch Operations Administration East', 'Branch Operations Administration West', 'Branch Operations Finance Staff East', 'Branch Operations Finance Staff West', 'Branch Outport Monitoring Class A East', 'Branch Outport Monitoring Class A West', 'Branch Outport Monitoring Class B West', 'Branch Outport Monitoring Class C East', 'Branch Outport Service Staff (Banjarmasin)', 'Branch Outport Service Staff (Bau-bau)', 'Branch Outport Service Staff (Berau)', 'Branch Outport Service Staff (Jayapura)', 'Branch Outport Service Staff (Makassar)', 'Branch Outport Service Staff (Medan)', 'Branch Outport Service Staff (Nunukan)', 'Branch Outport Service Staff (Samarinda)', 'Branch Outport Service Staff (Sorong)', 'Branch Outport Service Staff (Tual)', 'Branches Management Coordinator', 'Branches Management General Manager', 'Brand Communication Manager', 'Brand Manager', 'Building & Maintenance Administration', 'Building & Maintenance Assistant Manager', 'Building & Project Assistant Manager', 'Building & Project Supervisor', 'Building And Maintenance Supervisor', 'Building Maintenance', 'Building Maintenance Supervisor', 'Bulk Chartering Assistant Manager', 'Bulk Chartering Staff', 'Bulk Chartering Supervisor', 'Bulk Management Assistant Manager', 'Bulk Management Manager', 'Bulk Management Senior Manager', 'Bulk Operation Assistant Manager', 'Bulk Operation Staff', 'Bulk Operation Supervisor', 'Bunker And Overseas Purchasing Manager', 'Bunker Assistant Manager', 'Bunker Staff', 'Bunker Supervisor', 'Business Development Assistant Manager', 'Business Development Director', 'Business Development Manager', 'Business Development Manager (SBY)', 'Business Development Senior Manager', 'Business Development Staff', 'Business Development Supervisor', 'Business Performance', 'Business Performance Analyst', 'Business Process Excellence Manager', 'Business Process Owner Manager', 'Business Process Owner Senior Manager', 'Business Process Staff', 'CIC & Operations Adm. Staff', 'CNC Operator', 'CY Terminal Administration', 'CY Terminal Supervisor Berlian & Jamrud', 'CY Terminal Supervisor Mirah & Nilam', 'Calculating HOD', 'Call Center Branch Coordinator', 'Call Center Customer Service Executive', 'Call Center Customer Service Team Leader', 'Canteen Keeper', 'Care Customer Service Executive', 'Care Customer Service Team Leader', 'Cashier', 'Cashier (TPIL)', 'Cashier Branch Class A West', 'Cashier Branch Class B East', 'Cashier Branch Class B West', 'Cashier DPL', 'Cashier Jakarta Branch', 'Cashier TPIL SBY', 'Chief Administration Officer', 'Chief Commercial Officer', 'Chief Executive Officer', 'Chief Operation Officer', 'Chinese Desk Assistant Manager', 'Chrome', 'Civil Engineer', 'Claim & Insurance Commercial Administration', 'Claim & Insurance Commercial Staff', 'Claim & Insurance Commercial Supervisor', 'Claim & Insurance Fleet Staff', 'Claim And Insurance Fleet Supervisor', 'Claim And Legal', 'Claim Manager', 'Claim Officer', 'Cleaning Container', 'Cleaning Service', 'Cleaning Yard', 'Collector', 'Collector Branch Class B West', 'Collector Jakarta Branch', 'Collector TPIL JKT', 'Collector TPIL SBY', 'Command Center Manager', 'Commercial & Operation Staff', 'Commercial & Operations Cashier', 'Commercial & Operations Finance Assistant Manager', 'Commercial And Operations Finance Staff', 'Commercial And Operations Finance Supervisor', 'Commercial Continuous Improvement Manager', 'Commercial General Manager', 'Commercial Manager', 'Commercial Manager TPIL', 'Commercial Secretary', 'Company Medical Officer', 'Compliance Audit Assistant Manager', 'Compliance Audit Manager', 'Compliance Audit Supervisor', 'Consolidator Account Manager (AD-SBY)', 'Container Estimator', 'Container Inventory Control Administration', 'Container Inventory Control Administration Branch Class A West', 'Container Inventory Control Administration Branch Class B East', 'Container Inventory Control Administration Branch Class B West', 'Container Inventory Control Administration Branch Class C East', 'Container Inventory Control Administration Branch Class C West', 'Container Inventory Control Administration Branch Class D East', 'Container Inventory Control Administration Jakarta Branch', 'Container Inventory Control Analyst', 'Container Inventory Control Assistant Manager', 'Container Inventory Control Supervisor', 'Container Inventory Control Supervisor Jakarta Branch', 'Container Inventory Management Supervisor', 'Container Loading Conveyor Operator', 'Container Repair Advisor', 'Container Repair Supervisor', 'Container Surveyor Coordinator', 'Continuous Improvement Assistant', 'Continuous Improvement Assistant Manager', 'Continuous Improvement Coach', 'Continuous Improvement Coordinator Fleet And Support', 'Continuous Improvement Coordinator Jakarta Branch', 'Continuous Improvement Coordinator Operations And Commercial', 'Continuous Improvement Manager', 'Continuous Improvement Operations Manager', 'Continuous Improvement Staff', 'Continuous Improvement Supervisor', 'Control System Assistant Supervisor', 'Control System Electrician', 'Cook', 'Cook Assistant', 'Coordinator Driver Staff', 'Coordinator Driver Supervisor', 'Core Collector', 'Core Manager', 'Corp. Account & Value Added Service Analyst', 'Corp. Account & Value Added Service Manager', 'Corp. Account Customer Service Executive (Jakarta)', 'Corp. Account Customer Service Executive (Surabaya)', 'Corp. Account Customer Service Team Leader (Jakarta)', 'Corp. Account Customer Service Team Leader (Surabaya)', 'Corporate Account & Logistic Pricing Analyst', 'Corporate Account Coordinator (Jakarta)', 'Corporate Account Coordinator (Surabaya)', 'Corporate Account Manager', 'Corporate Account Manager (Jakarta)', 'Corporate Account Manager (Surabaya)', 'Corporate Account Sales Support', 'Corporate Affairs Director', 'Corporate Affairs Secretary', 'Corporate Communication Manager', 'Corporate Communication Supervisor', 'Corporate Communications Assistant Manager', 'Corporate Legal Strategic Manager', 'Cost & Budget Supervisor', 'Cost Finance And Administration Staff', 'Cost Finance Supervisor', 'Cost Finance and Administration Supervisor', 'Cost and Budget Supervisor', 'Courier', 'Courier TPIL JKT', 'Courier Warehouse', 'Crew and Ship Document Staff', 'Crewing', 'Customer Service Assistants', 'Customer Service Branch Class A East', 'Customer Service Branch Class A West', 'Customer Service Branch Class B East', 'Customer Service Branch Class B West', 'Customer Service Branch Class C East', 'Customer Service Branch Class C West', 'Customer Service Executive', 'Customer Service Jakarta Branch', 'Customer Service Manager', 'Customer Service Manager (Jakarta)', 'Customer Service Manager (SBY)', 'Customer Service Manager (Surabaya)', 'Customer Service Operation', 'Customer Service Operation Specialist', 'Customer Service Staff', 'Customer Service Supervisor Jakarta Branch', 'Customer Service Supervisor TPIL JKT', 'Customer Service Supervisor TPIL SBY', 'Customer Service Support (Jakarta)', 'Customer Services Assistant Manager', 'DPA Assistant', 'Data Analyst', 'Deputy Branch Manager TPIL', 'Deputy GM Business Partner', 'Deputy GM Inland Service', 'Deputy General Manager Corporate Affairs', 'Deputy General Manager DPL', 'Deputy General Manager Finance & Administration', 'Deputy General Manager Fleet', 'Deputy General Manager Human Capital', 'Deputy General Manager Human Resources Business Partner', 'Deputy General Manager Human Resources and General Affairs', 'Deputy General Manager Operations', 'Deputy General Manager Performance Excellence', 'Deputy Nautical Manager', 'Deputy Ship Personnel Manager', 'Deputy Technical Manager', 'Design and Budgeting Supervisor', 'Design and Building Control Staff', 'DevOps Engineer', 'DevOps Manager', 'DevSecOps Senior Manager', 'Development Staff', 'Digital Marketing Assistant Supervisor', 'Digital Marketing Staff', 'Direct Sourcing Assistant Supervisor', 'Direct Sourcing Staff', 'Direct Sourcing Supervisor', 'Discharge Customer Service Jakarta Branch', 'Discharge Customer Service Supervisor', 'Discharge Customer Service Supervisor Jakarta Branch', 'Discharge Customer Services Staff', 'Dispatch & Monitoring Coordinator', 'Dispatch & Monitoring Staff', 'Dispatch & Monitoring Staff (Jakarta)', 'Dock Monitoring Staff', 'Dock Monitoring Supervisor', 'Docking Electrical Maintenance Electrician', 'Document Assistant Manager', 'Document Control Coordinator', 'Document Core Staff', 'Document Core Supervisor', 'Document F/A Staff Branch Class A West', 'Document F/A Staff Branch Class B East', 'Document F/A Staff Branch Class B West', 'Document F/A Supervisor', 'Document Finance Staff', 'Document Finance Supervisor', 'Document Staff', 'Document Staff (Jakarta)', 'Document Staff Branch Class B East', 'Document Supervisor', 'Document and Goods Expedition Staff', 'Documentation Customer Service Executive', 'Dooring Coordinator (JKT)', 'Dooring Coordinator (SBY)', 'Dooring Dispatcher & Monitoring Staff', 'Dooring Driver', 'Dooring Driver (SBY)', 'Dooring Driver Support', 'Dooring Operation Staff', 'Dooring Supervisor (JKT)', 'Drafter', 'Drafter Staff', 'Driver Acquisition Staff', 'Driver Acquisition Supervisor', 'Driver Coordinator', 'Driver Jakarta Branch', 'Driver Kalianak', 'Driver Perak Barat', 'Driver TPIL', 'Driver Trailer', 'Driver Trailer Jakarta Branch', 'Driver Tronton', 'Driver Tronton Branch Class A West', 'Driver Tronton Branch Class B East', 'Driver Tronton Branch Class B West', 'Driver Tronton Branch Class C East', 'Driver Tronton Branch Class C West', 'Driver Tronton Branch Class D East', 'Driver Tronton Branch Class D West', 'Driver Tronton Branch Class E East', 'Driver Tronton Branch Class E West', 'Driver Truck', 'Dry Monitoring Staff', 'EMC Warehouse Coordinator', 'Electrical', 'Electrical Assistant Supervisor (DPL)', 'Electrical Project Planner & Coordinator', 'Electrical Staff', 'Electrical Supervisor', 'Electrical Supervisor DPL', 'Electrical Troubleshooting Assistant Supervisor', 'Electromagnetic Compatibility Warehouse Coordinator', 'Electromotor Assistant Supervisor', 'Electromotor Electrician', 'Electronic Staff', 'Electronic Supervisor', 'Employee Relationship Assistant Manager', 'Employer Branding & Organizational Development Manager', 'Employer Branding Manager', 'Employer Branding Staff', 'Employer Branding Supervisor', 'Engine Oil & Grease Purchasing Staff', 'Engineering Staff', 'Equipment Maintenance Center Assistant Manager', 'Equipment Maintenance Center Staff', 'Equipment Maintenance Center Supervisor', 'Estimator', 'Estimator Staff', 'Estimator Supervisor', 'Executive Officer', 'Exim Administration', 'Exim Administration and Operation Jakarta Branch', 'Exim Operation', 'Expatriat', 'Expedition & Delivery Staff', 'Expedition & Delivery Supervisor', 'Expedition Staff', 'F/A Staff Branch Class A West', 'F/A Staff Branch Class B East', 'F/A Staff Branch Class B West', 'F/A Staff Branch Class C East', 'F/A Staff Branch Class C West', 'F/A Staff Branch Class D East', 'FB Quality & Safety', 'Facility & Inventory Assistant Manager', 'Facility & Inventory Staff', 'Facility & Inventory Supervisor', 'Facility Administration', 'Facility Electrical Assistant Manager', 'Facility Electrical HOD', 'Fast Track Commercial', 'Fast Track Corporate Affairs', 'Fast Track Dok Pantai Lamongan', 'Fast Track Finance Accounting', 'Fast Track Human Capital & Corporate Affairs', 'Fast Track Operations', 'Fast Track Pelayaran Manalagi', 'Fax Operator', 'Feeder & SOC Account Manager', 'Feeder & SOC Sales', 'Feeder & SOC Sales Manager', 'Feeder & SOC Sales Supervisor', 'Feeder Operations', 'Feeder Operations (Jakarta)', 'Feeder Operations (Surabaya)', 'Feeder Operator', 'Field Man', 'Field Man Staff', 'Field Operation Building and Maintenance Staff', 'Field Operation Coordinator', 'Field Operation General Service Assistant', 'Field Operation Staff', 'Filing Support', 'Finance & Administration General Manager', 'Finance & Administration Head', 'Finance & Administration Senior Manager', 'Finance And Administration Director', 'Finance And Administration Head Branch Class A West', 'Finance And Administration Head Branch Class B East', 'Finance And Administration Head Branch Class B West', 'Finance And Administration Head Branch Class C East', 'Finance And Administration Head Branch Class C West', 'Finance And Administration Head Branch Class D East', 'Finance And Administration Head Branch Class D West', 'Finance And Administration Head Branch Class E East', 'Finance And Administration Head Branch Class E West', 'Finance And Administration Head Jakarta Branch', 'Finance And Administration Manager', 'Finance And Administration Staff Branch Class C East', 'Finance And Administration Staff Jakarta Branch (Fleet-Accounting-Treasury)', 'Finance And Adminstration Staff Branch Class C West', 'Finance And Personnel Head Branch Class C East', 'Finance And Personnel Head Branch Class C East (Bitung)', 'Finance And Personnel Head Branch Class D East', 'Finance And Personnel Head Branch Class D West', 'Finance And Personnel Head Branch Class E East', 'Finance And Personnel Head Branch Class E West', 'Finance Assistant Manager', 'Finance Assistant Manager TPIL JKT', 'Finance Assistant Manager TPIL SBY', 'Finance Manager', 'Finance Project Specialist', 'Finance Staff Branch Class D West', 'Finance Staff DPL', 'Finance and Administration Core Assistant Manager', 'Finance and Administration General Manager', 'Finance and Administration Staff', 'Fleet Administration', 'Fleet Administration Staff', 'Fleet Claim & Insurance Staff', 'Fleet Claim Survey Staff', 'Fleet Claim Survey Supervisor', 'Fleet Finance Assistant Manager', 'Fleet Finance Staff', 'Fleet Finance Supervisor', 'Fleet General Manager', 'Fleet Manager', 'Fleet Purchasing Administration Staff', 'Fleet Purchasing Assistant Manager', 'Fleet Purchasing Courier', 'Fleet Purchasing Staff', 'Fleet Purchasing Supervisor', 'Fleet Receiving Staff', 'Fleet Secretary', 'Fleet Secretary (Sr. Staff)', 'Flying Man', 'Flying Man (Jakarta)', 'Food & Beverage Supervisor', 'Food and Beverage Head', 'Food and Beverage Procurement Staff', 'Food and Beverage Supervisor', 'Foreman Ship Operation', 'Foreman Ship Operation Branch Class A East', 'Foreman Ship Operation Branch Class A West', 'Foreman Ship Operation Branch Class B East', 'Foreman Ship Operation Branch Class B West', 'Foreman Ship Operation Branch Class C East', 'Foreman Ship Operation Branch Class C West', 'Foreman Ship Operation Branch Class D East', 'Foreman Ship Operation Branch Class D West', 'Foreman Ship Operation Branch Class E East', 'Foreman Ship Operation Branch Class E West', 'Foreman Ship Operation Jakarta Branch', 'Foreman Ship Operation NPTI', 'Foreman Yard Operation (TSP/Mirah)', 'Foreman Yard Operation Branch Class A West', 'Foreman Yard Operation Branch Class D East', 'Foreman Yard Operation Staff', 'Foreman Yard Operations Branch Class C East', 'Foreman Yard Operations Branch Class C West', 'Forklift Operator', 'Forklift Operator Branch Class A West', 'Forklift Operator Branch Class B East', 'Forklift Operator Branch Class B West', 'Forklift Operator Branch Class C West', 'Forklift Operator Branch Class D East', 'Forklift Operator Jakarta Branch', 'Forklift Operator TSP/Mirah', 'GA Warehouse Assistant Manager', 'Gas & Compressor Operator', 'Gas dan Compressor Operator Staff', 'Gate Operator', 'General Accounting Affiliated Companies Assistant Manager', 'General Accounting Affiliated Companies Staff', 'General Accounting Affiliated Companies Supervisor', 'General Accounting Assistant Manager', 'General Accounting Coordinator', 'General Accounting Manager', 'General Accounting Staff', 'General Accounting Supervisor', 'General Affair Administration Staff', 'General Affair Core Staff', 'General Affair Core Supervisor', 'General Affair Internal Assistant Supervisor', 'General Affair Manager', 'General Affair Senior Staff Jakarta Branch', 'General Affair Staff', 'General Affair Staff Jakarta Branch', 'General Affair Supervisor DPL', 'General Affair Support Coordinator', 'General Affairs Officer Jakarta Branch', 'General Affairs Operations', 'General Affairs Representative', 'General Affairs Representative Staff', 'General Affairs Staff Kalianak', 'General Affairs Supervisor', 'General Affairs Supervisor Kalianak', 'General Affairs Warehouse Supervisor', 'General Affiar', 'General Ledger Accounting Staff', 'General Ledger Accounting Supervisor', 'General Legal Coordinator', 'General Legal Staff', 'General Legal Supervisor', 'General Manager Human Capital', 'General Manager TPIL Logistic', 'General Purchasing Assistant Manager', 'General Purchasing Staff', 'General Purchasing Supervisor', 'General Service Administration', 'General Service Assistant Manager', 'General Service Supervisor', 'General Warehouse Supervisor', 'General Welfare Staff', 'Generator Maintenance Electrician', 'Global Freight Manager', 'Group Leader Hull', 'Group Leader Ship Service', 'HOD Finance', 'HOD Human Resources & General Affair', 'HOD IT', 'HOD Piping & Carpenter', 'HOD Shipwright And Doc Master', 'HR & GA Officer', 'HR & GA Officer Branch Class A West', 'HR & GA Officer Branch Class B East', 'HR & GA Officer Branch Class B West', 'HR & GA Officer Cabang Banjarmasin', 'HR & GA Officer Commercial', 'HR & GA Officer Fleet', 'HR & GA Officer Kalianak', 'HR & GA Officer Perak Barat', 'HR Administrative Expertise', 'HR Administrative Manager', 'HR Administrative Staff', 'HR Administrative Supervisor', 'HR Change Agent Manager', 'HR Change Agent Officer', 'HR Employee Champion Manager', 'HR Manager', 'HR Manager (Acting)', 'HR Organizational Strategic Manager', 'HRBP Officer', 'HRIS Administration Supervisor', 'HRIS Manager', 'HRIS Staff', 'HRIS Supervisor', 'Haulage Administration', 'Haulage Administration Staff', 'Haulage Dispatcher', 'Haulage Dispatcher & Monitoring Staff', 'Haulage Driver', 'Haulage Officer', 'Haulage Operation Staff', 'Haulage Supervisor', 'Haulage Supervisor (Jakarta)', 'Head Office & Branch Finance Assistant Manager', 'Head Office Driver', 'Head Office Finance Assistant Manager', 'Head Office Finance Staff', 'Head Office Finance Supervisor', 'Head Office OB / OG', 'Head Office Phone Operator', 'Head of Commercial', 'Head of Corporate Communications', 'Head of Strategic Marketing', 'Health Safety & Environment Administration Staff', 'Health Safety & Environment Supervisor', 'Health Safety And Enviroment Manager', 'Health Safety And Environment Staff', 'Health Safety Environment', 'Health Safety Environment Assistant Manager', 'Health Safety Environment Officer', 'Health Safety Environment Staff', 'Health Safety Environment Supervisor', 'Health Safety and Environment Branch Staff', 'Health Safety and Environment Branch Supervisor', 'Health Safety and Environment Inspector', 'Health Safety and Environment Senior Staff', 'Health Safety and Environment Staff', 'Health Safety and Environment Supervisor', 'Health Safety and Environment Surabaya Staff', 'Health Safety and Environment Surabaya Supervisor', 'Heavy Equipment Administration Staff', 'Heavy Equipment Branch Mechanic', 'Heavy Equipment Document Control Staff', 'Heavy Equipment Electrician', 'Heavy Equipment Electrician & Radiator Supervisor', 'Heavy Equipment Electromotor & Radiator Assistant Supervisor', 'Heavy Equipment Head', 'Heavy Equipment Leakage Surveyor', 'Heavy Equipment Logistic Staff', 'Heavy Equipment Maintenance & Repair Assistant Manager', 'Heavy Equipment Maintenance & Repair Branch Assistant Manager', 'Heavy Equipment Maintenance & Repair Manager', 'Heavy Equipment Maintenance Assistant Supervisor', 'Heavy Equipment Maintenance Helper', 'Heavy Equipment Maintenance Mechanic', 'Heavy Equipment Maintenance Supervisor', 'Heavy Equipment Mechanic', 'Heavy Equipment Mechanic Assistant Supervisor', 'Heavy Equipment Mechanic Helper', 'Heavy Equipment Mechanic Supervisor', 'Heavy Equipment Operator', 'Heavy Equipment Operator (FL) Branch Class B West', 'Heavy Equipment Operator (FL) Branch Class C East', 'Heavy Equipment Operator (FL) Branch Class C West', 'Heavy Equipment Operator (FL) Jakarta Branch', 'Heavy Equipment Operator (RS) Jakarta Branch', 'Heavy Equipment Operator (SC) Branch Class C East', 'Heavy Equipment Operator (Tr) Branch Class B East', 'Heavy Equipment Operator (Tr) Branch Class C East', 'Heavy Equipment Operator Branch Class A West', 'Heavy Equipment Operator Branch Class B East', 'Heavy Equipment Operator Branch Class B West', 'Heavy Equipment Operator Branch Class C East', 'Heavy Equipment Operator Branch Class C West', 'Heavy Equipment Operator Branch Class D East', 'Heavy Equipment Operator Branch Class D West', 'Heavy Equipment Operator Branch Class E West', 'Heavy Equipment Operator Jakarta Branch', 'Heavy Equipment Operator Staff', 'Heavy Equipment Project Assistant Supervisor', 'Heavy Equipment Project Mechanic', 'Heavy Equipment Quality Control Staff', 'Heavy Equipment Repair & Maintenance Assistant Manager', 'Heavy Equipment Repair Assistant Manager', 'Heavy Equipment Repair Assistant Supervisor', 'Heavy Equipment Repair Helper', 'Heavy Equipment Repair Mechanic Supervisor', 'Heavy Equipment Repair Supervisor', 'Heavy Equipment Spare Part & Maintenance Supervisor', 'Heavy Equipment Trainee', 'Heavy Equipment Troubleshooting Helper', 'Heavy Equipment Troubleshooting Mechanic', 'Heavy Equipment Troubleshooting Mechanic Assistant Supervisor', 'Helper Fleet Troubleshoting Electrician', 'Helper Heavy Equipment Operator (RS)', 'Helper Mechanic', 'Helper Mechanic Branch Class A West', 'Helper Mechanic Branch Class B East', 'Helper Mechanic Branch Class C East', 'Helper Mechanic Branch Class C West', 'Helper Mechanic Branch Class D East', 'Helper Mechanic Heavy Equipment Maintenance', 'Helper Mechanic Heavy Equipment Repair', 'Helper Mechanic Machine Repair', 'Helper Mechanic Tools & Maintenance Support', 'Helper Panel Construction Electrician', 'Helper Shore Crane', 'Helper Shore Crane Branch Class D East', 'Helper Shore Crane Operator Branch Class C West', 'Helper Shore Crane Operator Branch Class D West', 'Helper Shore Crane Operator Branch Class E East', 'Helper Technician Injection Pump', 'Hiring Staff', 'Hiring Supervisor', 'Hull Administration', 'Hull Assistant Supervisor', 'Hull Fitter', 'Hull Fitter Assistant Supervisor', 'Hull Fitter Staff', 'Hull Fitter Supervisor', 'Hull HOD', 'Hull Outfit', 'Hull Pipping', 'Hull Pipping Staff', 'Hull Staff', 'Hull Supervisor', 'Hull Welder', 'Hull Welder Assistant Supervisor', 'Hull Welder Supervisor', 'Human Capital & Corporate Affairs Director', 'Human Capital & Corporate Affairs General Manager', 'Human Capital & Corporate Affairs Jakarta Manager', 'Human Capital & Corporate Affairs Secretary', 'Human Capital Assistant Manager', 'Human Capital Manager', 'Human Capital Supervisor', 'Human Capital and Corporate Affairs Jakarta Manager', 'Human Resource & General Affair Core Assistant Manager', 'Human Resource & General Affair Manager', 'Human Resource Core Staff', 'Human Resource Core Supervisor', 'Human Resource Supervisor', 'Human Resources & General Affair Representative', 'Human Resources Administration (Head Office)', 'Human Resources Administration (Jakarta Branch)', 'Human Resources Administration (Kalianak)', 'Human Resources Administration (Perak Barat)', 'Human Resources Administration Branch', 'Human Resources Administration Staff', 'Human Resources Administration Supervisor', 'Human Resources And General Affairs General Manager', 'Human Resources Business Partner Asst. Manager', 'Human Resources Business Partner Commercial Manager', 'Human Resources Business Partner Finance & Administration Manager', 'Human Resources Business Partner Finance & Administration and Operation Manager', 'Human Resources Business Partner Fleet Assistant Manager', 'Human Resources Business Partner Fleet Manager', 'Human Resources Business Partner HCCA, FAD, SDI, IA, BOD Manager', 'Human Resources Business Partner Human Capital & Corporate Affairs Manager', 'Human Resources Business Partner Human Capital & Finance and Administration Manager', 'Human Resources Business Partner Inland Service Manager', 'Human Resources Business Partner Jakarta Manager', 'Human Resources Business Partner Jakarta Staff', 'Human Resources Business Partner Jakarta Supervisor', 'Human Resources Business Partner Manager', 'Human Resources Business Partner Operation & FAD Manager', 'Human Resources Business Partner Operation & Finance and Administration Manager', 'Human Resources Business Partner Operation Assistant Manager', 'Human Resources Business Partner Operation East Area Supervisor', 'Human Resources Business Partner Operation Manager', 'Human Resources Business Partner Operation Staff', 'Human Resources Business Partner Operation Supervisor', 'Human Resources Business Partner Operation West Area Coordinator', 'Human Resources Business Partner Senior Manager', 'Human Resources Business Partner Staff', 'Human Resources Change Agent Assistant Manager', 'Human Resources Change Agent Manager', 'Human Resources Change Agent Supervisor', 'Human Resources Data Analyst Assistant Manager', 'Human Resources Data Analyst Staff', 'Human Resources Data Analyst Supervisor', 'Human Resources Development General Manager', 'Human Resources Information Staff', 'Human Resources Information System Analyst', 'Human Resources Information System Assistant Manager', 'Human Resources Information System Manager', 'Human Resources Information System Senior Staff', 'Human Resources Information System Staff', 'Human Resources Information System Supervisor', 'Human Resources Officer Jakarta Branch', 'Human Resources Staff', 'Human Resources System Officer', 'Human Resources and General Affair Supervisor', 'ISM Code Document Control Supervisor', 'IT Application Development Manager', 'IT General Manager', 'IT Network & Infrastructure Manager', 'IT Planning & Business Process Analyst Manager', 'IT Project Associate', 'IT Project Manager', 'IT Security Auditor', 'IT Security Specialist', 'IT Senior Manager', 'Improvement Project Officer', 'Inbound Sales Manager', 'Industrial & Employee Relation Staff', 'Industrial & Employee Relation Supervisor', 'Industrial & Employee Relations and Public Affairs Manager', 'Industrial & Employees Relations Manager', 'Industrial Relation Manager', 'Industrial Relation Officer', 'Industrial Relation Staff', 'Industrial Relation Supervisor', 'Industrial Relations Manager', 'Information Security Specialist', 'Information Technology Assistant Manager', 'Information Technology Supervisor', 'Information Technology System Manager', 'Inland Maintenance Helper', 'Inland Service A/P Staff', 'Inland Service A/R Staff', 'Inland Service Account Payable Staff', 'Inland Service Account Receivable Staff', 'Inland Service Administration', 'Inland Service Administration Staff', 'Inland Service Administration Supervisor', 'Inland Service Administrator', 'Inland Service Administrator (Jakarta)', 'Inland Service Assistant Manager', 'Inland Service Assistant Manager (Jakarta)', 'Inland Service Assistant Supervisor', 'Inland Service Business Partner', 'Inland Service Deputy General Manager', 'Inland Service Development Assistant Manager', 'Inland Service Document Outbond', 'Inland Service Document Outbond Staff', 'Inland Service Document Supervisor', 'Inland Service Dooring Supervisor', 'Inland Service Haulage Supervisor', 'Inland Service Jakarta Manager', 'Inland Service Kalimantan Administration Staff', 'Inland Service Kalimantan Administration Supervisor', 'Inland Service Kalimantan Manager', 'Inland Service Kalimantan Staff', 'Inland Service Kalimantan Supervisor', 'Inland Service Key Account Manager', 'Inland Service Manager', 'Inland Service National Manager', 'Inland Service Papua Administration Staff', 'Inland Service Papua Administration Supervisor', 'Inland Service Papua Manager', 'Inland Service Papua Staff', 'Inland Service Papua Supervisor', 'Inland Service Pricing Assistant Manager', 'Inland Service Pricing Assistant Supervisor', 'Inland Service Pricing Staff', 'Inland Service Pricing Supervisor', 'Inland Service Sulawesi Administration Staff', 'Inland Service Sulawesi Administration Supervisor', 'Inland Service Sulawesi Manager', 'Inland Service Sulawesi Staff', 'Inland Service Sulawesi Supervisor', 'Inland Service Sumatera Administration Staff', 'Inland Service Sumatera Administration Supervisor', 'Inland Service Sumatera Manager', 'Inland Service Sumatera Staff', 'Inland Service Sumatera Supervisor', 'Inland Service Supervisor', 'Inland Service Supervisor (Jakarta)', 'Inland Service Surabaya Manager', 'Inland Tallyman', 'Inland Tallyman (Jakarta)', 'Insurance & Claim Assistant Manager', 'Insurance & Claim Corporate Account', 'Insurance & Claim Corporate Account Staff', 'Insurance & Claim Fleet', 'Insurance & Claim Retail & Liner', 'Insurance & Claim Retail & Liner Staff', 'Insurance & Outsource Staff', 'Insurance Commercial Administration', 'Insurance Commercial Staff', 'Insurance Commercial Supervisor', 'Insurance Fleet Supervisor', 'Insurance Sales', 'Insurance Sales & Claim Manager', 'Insurance Sales Manager', 'Interchange Surveyor Coordinator', 'Interchange Surveyor Staff', 'Interchange Surveyor Supervisor', 'Internal Audit Assistant Manager', 'Internal Audit General Manager', 'Internal Audit Manager', 'Internal Audit Staff', 'Internal Audit Staff DPL', 'Internal Audit Supervisor', 'Internal Communication Executive', 'International Business Senior Manager', 'International Trade Cust. Service & Operation Manager', 'International Trade Cust. Service Executive (Jakarta)', 'International Trade Cust. Service Executive (Surabaya)', 'International Trade Cust. Service Team Leader (Jakarta)', 'International Trade Operations Team Leader', 'International Trade Ops. Executive (Jakarta)', 'International Trade Ops. Executive (Surabaya)', 'International Trade Sales Support (Jakarta)', 'Internship', 'Inventory Staff', 'Investigator & Security Coach', 'Investigator & Security Manager', 'Investigator & Security Officer', 'Investigator & Security Staff', 'Investigator & Security Supervisor', 'Investigator And Coaching Security Coordinator', 'Investment Junior Associate', 'IoT System Development Specialist', 'Jakarta Account Development Manager', 'Jakarta Branch Manager', 'Jakarta Deputy Branch Manager', 'Jakarta Feeder & SOC Sales', 'Jakarta International Trade Sales Support', 'Jakarta Operation Manager', 'Job Management', 'Junior Associate ESG Specialist', 'Junior Data Analyst', 'Junior Software Engineer', 'Kerani', 'Key Account Assistant Manager', 'Key Account Supervisor', 'Koordinator Armada Assistant Supervisor', 'Koordinator Armada Supervisor', 'Koordinator Driver', 'Koperasi Staf', 'Koperasi Supervisor', 'Land Acquisition', 'Learning And Development Manager', 'Learning And Development Staff', 'Learning And Development Supervisor', 'Learning Organization Staff', 'Learning Staff', 'Legal & Claim Manager', 'Legal & Land Acquisition Assistant Manager', 'Legal Assistant Manager', 'Legal Compliance Assistant Manager', 'Legal Compliance Manager', 'Legal Officer', 'Legal Supervisor', 'Legal, Public Affairs & Land Acquisition Assistant Manager', 'Loading Customer Service Jakarta Branch', 'Loading Customer Service Supervisor (Kalimantan dan Sumatra)', 'Loading Customer Service Supervisor (Papua dan Sulawesi)', 'Loading Customer Service Supervisor Jakarta Branch', 'Loading Customer Services Staff (Kalimantan dan Sumatra)', 'Loading Customer Services Staff (Papua dan Sulawesi)', 'Logistic Assistant Supervisor', 'Logistic Container Repair Staff', 'Logistic Container Repair Supervisor', 'Logistic Manager', 'Logistic Operation Branch Class B West', 'Logistic Operation Coordinator', 'Logistic Operation Worker', 'Logistic Operations Coordinator', 'Logistic Repair Container Staff', 'Logistic Repair Container Supervisor', 'Logistic and Repair Staff', 'Logistic and Vehicle Staff', 'Loss Prevention Assistant Manager', 'MDP Finance & Accounting', 'MT Executive', 'Machine Repair Assistant Supervisor', 'Machine Repair Supervisor', 'Machinery HOD', 'Machinery Mechanic Deck', 'Machinery Mechanic Deck Assistant Supervisor', 'Machinery Mechanic Deck Staff', 'Machinery Mechanic Deck Supervisor', 'Machinery Mechanic Shop', 'Machinery Mechanic Shop Assistant Supervisor', 'Machinery Mechanic Shop Staff', 'Machinery Quality Control Staff', 'Machinery Quality Control Supervisor', 'Machining', 'Maintenance & Inventory Supervisor', 'Maintenance & Repair Purchasing Staff', 'Maintenance & Repair Purchasing Supervisor', 'Maintenance & Repair Supervisor', 'Maintenance Coordinator', 'Maintenance Staff', 'Maker Technician', 'Maker Technician Assistant', 'Management Accounting Affiliated Companies Assistant Manager', 'Management Accounting Affiliated Companies Staff', 'Management Accounting Affiliated Companies Supervisor', 'Management Accounting Assistant Manager', 'Management Accounting Branch Staff', 'Management Accounting Branch Supervisor', 'Management Accounting Head Office Staff', 'Management Accounting Head Office Supervisor', 'Management Accounting Manager', 'Management Executive', 'Management Trainee', 'Manning Officer', 'Manning Supervisor', 'Marketing & Communication Manager', 'Marketing Administration', 'Marketing Analyst Coordinator', 'Marketing And Customer Services Head', 'Marketing Assistant Manager (Kalimantan dan Sumatera)', 'Marketing Assistant Manager (Papua)', 'Marketing Assistant Manager Inbound And SOC', 'Marketing Assistant Manager Jakarta Branch', 'Marketing Branch Class A East', 'Marketing Branch Class A West', 'Marketing Branch Class B East', 'Marketing Branch Class B West', 'Marketing Branch Class C East', 'Marketing Branch Class C West', 'Marketing Branch Class D East', 'Marketing Branch Class D West', 'Marketing Executive (Papua)', 'Marketing Executive (Sulawesi dan Maluku)', 'Marketing Executive (Sumatera dan Kalimantan)', 'Marketing Executive Branch Class A East', 'Marketing Executive Branch Class A West', 'Marketing Executive Branch Class B East', 'Marketing Executive Branch Class B West', 'Marketing Executive Branch Class C East', 'Marketing Executive Branch Class C West', 'Marketing Executive Branch Class D East', 'Marketing Executive Branch Class D West', 'Marketing Executive Branch Class E East', 'Marketing Executive Branch Class E West', 'Marketing Executive Coordinator (Papua)', 'Marketing Executive Coordinator (Reefer & Exim)', 'Marketing Executive Coordinator (Sulawesi dan Maluku)', 'Marketing Executive Coordinator (Sumatera dan Kalimantan)', 'Marketing Executive Coordinator (Trucking, Transhipment and Inbound)', 'Marketing Executive Coordinator Branch Class A West', 'Marketing Executive Coordinator Branch Class B East', 'Marketing Executive Coordinator Branch Class B West', 'Marketing Executive Coordinator Branch Class C East', 'Marketing Executive Coordinator Branch Class C West', 'Marketing Executive Coordinator Branch Class D West', 'Marketing Executive Coordinator Jakarta Branch', 'Marketing Executive Exim Jakarta Branch', 'Marketing Executive Inbound', 'Marketing Executive Inbound Coordinator', 'Marketing Executive Jakarta Branch', 'Marketing Executive Makassar', 'Marketing Executive Merauke', 'Marketing Executive Palembang', 'Marketing Executive Pontianak', 'Marketing Executive Surabaya', 'Marketing Executive TPIL Jakarta', 'Marketing Executive Tarakan', 'Marketing Executives', 'Marketing Exim Operation Jakarta Branch', 'Marketing Head', 'Marketing Head Branch Class A East', 'Marketing Head Branch Class A West', 'Marketing Head Branch Class B East', 'Marketing Head Branch Class B West', 'Marketing Head Branch Class C East', 'Marketing Head Branch Class D West', 'Marketing Head Jakarta Branch', 'Marketing Head Jayapura', 'Marketing Manager', 'Marketing Manager (Return Cargo, Reefer & Exim)', 'Marketing Manager (Sulawesi dan Maluku)', 'Marketing Manager (Sumatera dan Kalimantan)', 'Marketing Manager DPL', 'Marketing Manager Inbound & SOC', 'Marketing Specialist', 'Marketing Staff', 'Marketing Staff DPL', 'Marketing Supervisor', 'Marketing Supervisor .', 'Markonis', 'Mechanic', 'Mechanic Assistant Supervisor', 'Mechanic Branch Class A West', 'Mechanic Branch Class B East', 'Mechanic Branch Class B West', 'Mechanic Branch Class C East', 'Mechanic Branch Class C West', 'Mechanic Branch Class D East', 'Mechanic Branch Class D West', 'Mechanic Branch Class E East', 'Mechanic Branch Class E West', 'Mechanic DPL', 'Mechanic Electronic', 'Mechanic Heavy Equipment Maintenance', 'Mechanic Heavy Equipment Repair', 'Mechanic Jakarta Branch', 'Mechanic Machine Repair', 'Mechanic Navigation', 'Mechanic Purifier', 'Mechanic Staff', 'Mechanic Supervisor', 'Mechanic Supervisor Branch Class A West', 'Mechanic Supervisor Branch Class B East', 'Mechanic Supervisor Branch Class B West', 'Mechanic Supervisor Branch Class C East', 'Mechanic Supervisor Jakarta Branch', 'Mechanic Support', 'Mechanic Trailer Maintenance', 'Mechanic maintenance Heavy Equipment Group Leader', 'Mechanical', 'Mechanical & Electrical Staff', 'Mechanical Advisor', 'Mechanical Electrical Engineer', 'Mechanical Electrical Staff', 'Monitoring Assistant Supervisor', 'Monitoring Staff', 'Monitoring Supervisor', 'NDT Staff', 'NDT Supervisor', 'National Sales Manager', 'National Trade Manager', 'Nautical Advisor', 'Nautical Manager', 'Navigation Assistant Supervisor', 'Network Administration Supervisor', 'Network Administrator', 'Network Specialist', 'Network Specialist Assistant', 'Next Generation Leader', 'Non Destructive Test Manager', 'Non Destructive Test Operator', 'Non Destructive Test Staff', 'Non Destructive Test Supervisor', 'Non Maintenance & Repair Purchasing Staff', 'Non Maintenance & Repair Purchasing Supervisor', 'OB/OG Branch Class D West', 'OSSO Assistant Supervisor', 'OSSO Document Staff', 'OSSO Monitoring Staff', 'OSSO Supervisor', 'Office Boy', 'Office Boy (JAPFA)', 'Office Boy TPIL SBY', 'Office Facility Supervisor', 'Office Girl', 'Office Girl (Kalianak)', 'Office Girl (Kantor Pusat)', 'Office Girl (Perak Barat)', 'Office Girl (TSP)', 'Office Girl Branch Class A West', 'Office Girl Branch Class B West', 'Office Girl Branch Class D West', 'Office Of Strategic Management Staff', 'Office Supply Staff', 'Office of Strategy Management Manager', 'Oil Analyst', 'Operation Administration Branch Class C West', 'Operation Advisor', 'Operation Audit Assistant Manager', 'Operation Business Partner Staff', 'Operation Business Partner Supervisor', 'Operation Deputy General Manager East', 'Operation Director', 'Operation General Manager', 'Operation Head', 'Operation Head Assistant Samarinda', 'Operation Head Branch Class A East', 'Operation Head Branch Class A West', 'Operation Head Branch Class B East', 'Operation Head Branch Class B West', 'Operation Head Branch Class C East', 'Operation Head Branch Class C West', 'Operation Head Branch Class C West (Palembang)', 'Operation Head Branch Class D East', 'Operation Head Branch Class D West', 'Operation Logistic Worker', 'Operation Officer Ambon', 'Operation Officer Opin', 'Operation Regional Coordinator', 'Operation Regional East Assistant Manager', 'Operation Regional West', 'Operation Staff', 'Operations & Administration Core Manager', 'Operations Administration', 'Operations Administration Branch Class B East', 'Operations Administration Branch Class C East', 'Operations Administration Branch Class D West', 'Operations Assistant', 'Operations Assistant Manager', 'Operations Assistants', 'Operations Business Partner ISD', 'Operations Deputy GM East', 'Operations Deputy GM West', 'Operations Executive', 'Operations Head Branch Class D East', 'Operations Head Branch Class D West', 'Operations ISD Business Partner Ast. Manager', 'Operations Manager', 'Operations Officer Branch Class D East', 'Operations Officer Branch Class E East', 'Operations Officer Branch Class E West', 'Operations Secretary', 'Operations Staff Branch Class E East', 'Operations Staff Branch Class E West', 'Operations Staff Jakarta Branch', 'Operations Supervisor', 'Operations Supervisor TPIL', 'Operator', 'Operator Mesin Bending', 'Operator Mesin Winch', 'Operator Supervisor', 'Organisational Development Asst. Manager', 'Organizational & Talent Management Manager', 'Organizational Development Assistant Manager', 'Organizational Development Manager', 'Organizational Development Supervisor', 'Outport Assistant Manager', 'Outport Branch Assistant Manager (SBY)', 'Outport Branch Assistant Supervisor (JKT)', 'Outport Branch Assistant Supervisor (SBY)', 'Outport Branch Staff (JKT)', 'Outport Branch Staff (SBY)', 'Outport Branch Supervisor (JKT)', 'Outport Branch Supervisor (SBY)', 'Outport Service Branch Class A East', 'Outport Service Branch Class A West', 'Outport Service Branch Class B East', 'Outport Service Branch Class B West', 'Outport Service Branch Class C East', 'Outport Service Branch Class C West', 'Outport Service Branch Class D East', 'Outport Service Branch Class D West', 'Outport Service Branch Class E West', 'Outport Service Staff', 'Outport Service Supervisor', 'Outsource Acquisition Specialist', 'Overseas Purchasing Administration Staff', 'Overseas Purchasing Assistant Manager', 'Overseas Purchasing Manager', 'Overseas Purchasing Staff', 'Overseas Purchasing Supervisor', 'Painting Staff', 'Panel Construction Electrician', 'People Development Assistant Manager', 'People Development Staff', 'Performance Excellence Commercial & Inland Service Department Manager', 'Performance Excellence Commercial & Inland Service Department Staff', 'Performance Excellence Commercial & Inland Services Department Coordinator', 'Performance Excellence Commercial & Inland Services Department Manager', 'Performance Excellence Deputy GM', 'Performance Excellence Fleet Manager', 'Performance Excellence Human Capital & Finance Administration Division Staff', 'Performance Excellence Human Capital & Finance and Administration Division Coordinator', 'Performance Excellence Human Capital & Finance and Administration Division Manager', 'Performance Excellence Human Capital & Finance and Administration Division Supervisor', 'Performance Excellence Operations East Area Coordinator', 'Performance Excellence Operations East Area Manager', 'Performance Excellence Operations East Area Supervisor', 'Performance Excellence Operations Strategic East Area Manager', 'Performance Excellence Operations Strategic West Area Manager', 'Performance Excellence Operations West Area Manager', 'Performance Excellence Operations West Area Supervisor', 'Performance Excellence West Area Senior Manager', 'Performance Management Officer', 'Personnel Admin Supervisor', 'Personnel Administration Jakarta Branch', 'Personnel And General Affairs Staff Branch Class A West', 'Personnel And General Affairs Staff Branch Class B West', 'Personnel Assistant Manager (Branches)', 'Personnel Assistant Manager (Surabaya)', 'Personnel Manager', 'Personnel Manager (Branches)', 'Personnel Manager (Surabaya)', 'Personnel Officer', 'Personnel Staff', 'Personnel Supervisor', 'Personnel Supervisor (Branches)', 'Personnel Supervisor (Surabaya)', 'Pimpinan Proyek', 'Pipping', 'Pipping Assistant Supervisor', 'Pjs. Administrative Deputy General Manager Corporate Affairs', 'Port Captain', 'Port Captain Administration Jakarta Branch', 'Port Captain Administration Staff', 'Port Captain Assistant Jakarta Branch', 'Port Captain Jakarta Branch', 'Port Clearance Administration Staff', 'Port Clearance Arrival Staff', 'Port Clearance Crew Document Staff', 'Port Clearance Departure Staff', 'Port Clearance Finance Control Supervisor', 'Port Clearance Meeting Staff', 'Port Clearance Ship Document Staff', 'Port Clearance Supervisor', 'Pricing Administration', 'Pricing Analyst', 'Pricing Analyst (Jakarta)', 'Pricing Analyst (Kalimantan & Sumatera)', 'Pricing Analyst (Sulawesi & Papua)', 'Pricing Analyst Coordinator (Kalimantan & Sumatera)', 'Pricing Analyst East', 'Pricing Analyst Officer (Sulawesi & Papua)', 'Pricing Analyst West', 'Process Excellence Manager', 'Process Excellence Senior Manager', 'Process Improvement Assistant Manager', 'Process Improvement Manager', 'Procurement Head', 'Procurement Manager', 'Procurement Staff', 'Procurement Staff Jakarta Branch', 'Procurement Supervisor', 'Product Manager', 'Production Director', 'Production HOD', 'Production Manager', 'Production Planning Control Supervisor', 'Production Staff', 'Production Technical Supervisor', 'Program Engineer Officer', 'Programmer', 'Programmer (Staff)', 'Programmer Staff', 'Programmer Staff DPL', 'Programmer Supervisor', 'Programmer Supervisor DPL', 'Project & Building Maintenance Administration Staff', 'Project & Building Maintenance Administration Supervisor', 'Project & Building Maintenance Assistant Manager', 'Project & Building Maintenance Coordinator', 'Project & Building Maintenance Manager', 'Project & Building Maintenance Staff', 'Project & Building Maintenance Supervisor', 'Project Admin', 'Project Assistant Manager', 'Project Associate', 'Project Business Development Manager', 'Project Civil Engineer', 'Project Coordinator Manager', 'Project Improvement Staff', 'Project Manager', 'Project Planner Assistant Supervisor', 'Project Planner Staff', 'Project Purchasing Staff', 'Project Purchasing Supervisor', 'Project Research', 'Project Supervisor', 'Project and Building Maintenance Manager', 'Public Affair Manager', 'Public Affair Project Assistant Manager', 'Public Affair, Industrial & Employee Relation Manager', 'Public Affair, Industrial & Employee Relation National Manager', 'Public Affairs & Corporate Communication Staff', 'Public Affairs & Relation Manager', 'Public Affairs And Security Manager', 'Public Affairs Staff', 'Public Affairs, Industrial & Employees Relations Manager', 'Puchasing Coordinator', 'Purchasing Administration Staff', 'Purchasing Administration Staff DPL', 'Purchasing Administration Supervisor', 'Purchasing Assistant Manager', 'Purchasing Assistant Supervisor', 'Purchasing Asst. Supervisor', 'Purchasing Fresh Water & Fleet Workshop Staff', 'Purchasing Staff', 'Purchasing Staff (HE, DPL, UMUM)', 'Purchasing Supervisor', 'Purchasing Support', 'QHSE Administration Staff', 'QHSE Advisor', 'QHSE Manager', 'Quality Assurance', 'Quality Assurance & Continuous Improvement Assistant Manager', 'Quality Assurance Assistant Supervisor', 'Quality Assurance Container Supervisor', 'Quality Assurance Manager', 'Quality Assurance Staff', 'Quality Assurance Supervisor', 'Quality Control', 'Quality Control HOD', 'Quality Control Staff', 'Quality Health Safety & Environment Manager', 'Quality Health Safety And Environment Manager', 'Quality Health Safety And Environment National Manager', 'Quality Health, Safety And Enviroment Head', 'Quality Management Administration', 'Quality Management Assistant Manager', 'Quality Management Manager', 'Quantity Surveyor', 'RTG Maintenance Mechanic', 'RTG Maintenance Supervisor', 'RTG Operator', 'Radio Officer', 'Reachstacker Operator', 'Reachstacker Operator (TSP/Mirah)', 'Reachstacker Operator Branch Class A West', 'Receiving Staff', 'Receptionist & Telephone Operator', 'Receptionist TPIL JKT', 'Receptionist TPIL SBY', 'Recruitment Crew', 'Recruitment Crew & Manning Assistant Manager', 'Recruitment Crew & Manning Ast. Manager', 'Recruitment Manager', 'Recruitment Manager (Jakarta)', 'Recruitment Staff', 'Recruitment Supervisor', 'Reefer & Project Account Manager', 'Reefer & Project Staff', 'Reefer & Project Support', 'Reefer Container Monitoring Staff', 'Regional East Advisor', 'Regional Manager - East', 'Regional Manager - West', 'Regional Marketing Manager', 'Regional Sales Manager', 'Regional West Advisor', 'Regulatory Affairs Staff', 'Rendal Administration', 'Rendal Assistant Manager', 'Rendal HOD', 'Rendal Manager DPL', 'Repair Container Administration', 'Repair Container Assistant Manager', 'Repair Container Helper', 'Repair Container Logistic', 'Repair Container Supervisor', 'Repair Container Supervisor Jakarta Branch', 'Repair Container Support Staff', 'Repair Container Surveyor Staff', 'Repair Container Welder', 'Repair Planner Staff', 'Revenue Finance And Administration Supervisor', 'Revenue Finance Supervisor', 'Route & Capacity', 'Route & Capacity Manager', 'Route & Capacity Trade Manager', 'SDI Support Coordinator', 'SDI Support Staff', 'SPARTAN Executive Trainee CMD', 'SPARTAN Executive Trainee FAD', 'SPARTAN Executive Trainee Fleet', 'SPARTAN Fleet Trainee', 'SPARTAN Management Trainee HCCA', 'SPARTAN Management Trainee OPS', 'SPARTAN Mechanic Trainee HVE', 'Safety Inspector', 'Safety Officer', 'Sales Advisor', 'Sales Executive (Jakarta)', 'Sales Executive (Surabaya)', 'Sales Officer (Jakarta)', 'Sales Staff', 'Sales Support', 'Sales Support (AD-JKT)', 'Sales Support (AD-SBY)', 'Sales Support (AM-JKT)', 'Sales Support (AM-SBY)', 'Sales Trainee', 'Salvage Sales and Disposal', 'Salvage Sourcing and Valuation', 'Secretary', 'Secretary DPL', 'Secretary Staff', 'Secretary to Commercial General Manager', 'Secretary to President Director', 'Security', 'Security Administration', 'Security Assistant Manager', 'Security Coordinator', 'Security Guard', 'Security Guard Branch Class B East', 'Security Guard Branch Class B West', 'Security Guard Branch Class C East', 'Security Guard Branch Class C West', 'Security Guard Branch Class D East', 'Security Guard Branch Class E East', 'Security Guard Jakarta Branch', 'Security Guard Matrako', 'Security Supervisor', 'Senior Digital Marketing Specialist', 'Senior Field Operation Building And Maintenance', 'Senior Marketing Executive', 'Senior Pimpinan Proyek', 'Senior Port Captain', 'Senior Secretary', 'Senior Software Engineer', 'Senior Superintendent', 'Senior Superintendent Jakarta Branch', 'Senior Trainer', 'Sensor & Electrical Leader', 'Service Quality Assurance Assistant Manager', 'Ship Classification Clearance', 'Ship Classification Clearance Officer', 'Ship Clearance Administration', 'Ship Clearance Administration Jakarta Branch', 'Ship Clearance And Logistic Staff Branch Class A East', 'Ship Clearance And Logistic Staff Branch Class A West', 'Ship Clearance And Logistic Staff Branch Class B East', 'Ship Clearance And Logistic Staff Branch Class B West', 'Ship Clearance And Logistic Staff Branch Class C East', 'Ship Clearance And Logistic Staff Branch Class C West', 'Ship Clearance And Logistic Staff Branch Class D East', 'Ship Clearance And Logistic Staff Branch Class D West', 'Ship Clearance And Trailer Supervisor', 'Ship Clearance Arrival Staff', 'Ship Clearance Assistant Supervisor Jakarta Branch', 'Ship Clearance Departure Staff', 'Ship Clearance Meeting Staff', 'Ship Clearance Staff', 'Ship Clearance Staff Branch Class A West', 'Ship Clearance Staff Branch Class B East', 'Ship Clearance Staff Branch Class B West', 'Ship Clearance Staff Branch Class C East', 'Ship Clearance Staff Branch Class C West', 'Ship Clearance Staff Branch Class D East', 'Ship Clearance Staff Branch Class D West', 'Ship Clearance Staff Branch Class E East', 'Ship Clearance Staff Jakarta Branch', 'Ship Clearance Supervisor', 'Ship Clearance Supervisor Jakarta Branch', 'Ship Clearance and CY Terminal Supervisor', 'Ship Clearence And Logistic Branch Class A West', 'Ship Clearence And Logistic Branch Class B West', 'Ship Company Policies Supervisor', 'Ship Crew Training & Development Supervisor', 'Ship Document Keeper', 'Ship Document Officer', 'Ship Learning & Development Assistant Manager', 'Ship Learning & Development Ast. Manager', 'Ship Learning & Development Manager', 'Ship Logistic Staff', 'Ship Operation Administration', 'Ship Operation Administration Jakarta Branch', 'Ship Operation Assistant Manager', 'Ship Operation Assistant Supervisor', 'Ship Operation Assistant Supervisor Jakarta Branch', 'Ship Operation Assistant Supervisor NPTI', 'Ship Operation Manager', 'Ship Operation Staff', 'Ship Operation Staff Jakarta Branch', 'Ship Operation Supervisor', 'Ship Operation Supervisor Branch Class A West', 'Ship Operation Supervisor Branch Class B East', 'Ship Operation Supervisor Branch Class B West', 'Ship Operation Supervisor Branch Class C East', 'Ship Operation Supervisor Branch Class C West', 'Ship Operation Supervisor Branch Class D East', 'Ship Operation Supervisor Jakarta Branch', 'Ship Operations Head', 'Ship Personnel', 'Ship Personnel Administration', 'Ship Personnel Administration & Remuneration', 'Ship Personnel Management Manager', 'Ship Personnel Manager', 'Ship Personnel Supervisor', 'Ship Service', 'Ship Service Assistant Supervisor', 'Ship Service Staff', 'Ship Superintendent Staff', 'Shipwright Airbag', 'Shipwright Cleaning', 'Shipwright Cleaning Staff', 'Shipwright Dock', 'Shipwright Dock Assistant Supervisor', 'Shipwright Dock Staff', 'Shipwright Painting Supervisor', 'Shipwright Scafolding Staff', 'Shipwright Supervisor', 'Shore Crane Operator', 'Shore Crane Operator Branch Class C West', 'Shore Crane Operator Branch Class D East', 'Shore Crane Operator Branch Class D West', 'Shore Crane Operator Branch Class E East', 'Shore Crane Operator Branch Class E West', 'Shore Crane Operator Jakarta Branch', 'Side Loader Operator', 'Software Development & Business Process Analyst Manager', 'Software Development & Business Process Analyst Senior Manager', 'Software Engineer', 'Solution Customer Service Executive', 'Solution Customer Service Team Leader', 'Sounding Staff', 'Sounding Staff Jakarta Branch', 'Spare Part Repair Assistant Supervisor', 'Spare Part Repair Operator', 'Spare Part Repair Operator Heavy Equipment', 'Spare Part Repair Supervisor', 'Spare Part Repair Welder', 'Standardization Project Manager', 'Standardization Project Senior Manager', 'Stock Coordinator', 'Stock Staff', 'Stock Supervisor', 'Strategic & Service Excellence Deputy GM', 'Strategic Excellence Senior Manager', 'Strategy Development & Implementation Support Coordinator', 'Strategy Development & Implementation Support Staff', 'Strategy Development & Implementation Support Supervisor', 'Strategy Development and Implementation General Manager', 'Strategy Management Coordinator', 'Superintendent', 'Superintendent Jakarta Branch', 'Supervisor Planner', 'Support Technician', 'Surveyor Container', 'Surveyor Container Assistant', 'Surveyor Container Jakarta Branch', 'Surveyor Logistic Container Repair', 'System Administrator', 'System Analyst Supervisor', 'TPS B3 & Scrab Yard Group Leader', 'Talent Acquisition & Employer Branding Manager', 'Talent Acquisition Assistant Manager', 'Talent Acquisition Ast. Manager', 'Talent Acquisition Manager', 'Talent Acquisition Staff', 'Talent Acquisition Supervisor', 'Talent Assessment Staff', 'Talent Assessor', 'Talent Development Manager', 'Talent Development Supervisor', 'Talent Management & Development Assisstant Manager', 'Talent Management & Development Supervisor', 'Talent Management & Org. Development Manager', 'Talent Management Administration', 'Talent Management Supervisor', 'Tallyman Interchange', 'Tallyman Interchange Ambon', 'Tallyman Interchange Branch Class A West', 'Tallyman Interchange Branch Class B East', 'Tallyman Interchange Branch Class B West', 'Tallyman Interchange Branch Class C East', 'Tallyman Interchange Branch Class C West', 'Tallyman Interchange Branch Class D East', 'Tallyman Interchange Jakarta Branch', 'Tallyman Loading Discharge', 'Tallyman Loading Discharge (TPS)', 'Tallyman Loading Discharge (TSP/Mirah)', 'Tallyman Loading Discharge Branch Class A West', 'Tallyman Loading Discharge Branch Class B East', 'Tallyman Loading Discharge Branch Class B West', 'Tallyman Loading Discharge Branch Class C East', 'Tallyman Loading Discharge Branch Class C West', 'Tallyman Loading Discharge Branch Class D East', 'Tallyman Loading Discharge Branch Class D West', 'Tallyman Loading Discharge Branch Class E East', 'Tallyman Loading Discharge Branch Class E West', 'Tallyman Loading Discharge Jakarta Branch', 'Tax Assistant Manager', 'Tax Staff', 'Tax Supervisor', 'Technical Administration Staff', 'Technical Administration Staff Jakarta Branch', 'Technical Administrator Assistant', 'Technical Dry Docking Administration', 'Technical Manager', 'Technical Performance Management (Heavy Equipment) Specialist', 'Technical Project Officer', 'Technical Purchasing Supervisor', 'Technical Support Assistant Manager', 'Technical Support Staff', 'Technical Support Staff Jakarta Branch', 'Technical Support Supervisor', 'Telephone Operator', 'Telephone Operator (Perak Barat)', 'Telephone Operator Branch Class A West', 'Telephone Operator Branch Class B West', 'Telephone Operator TPIL SBY', 'Telesales Executive', 'Ticketing Staff', 'Ticketing and Hotel Staff', 'Time Keeper', 'Tool & Maintenance Support Assistant Supervisor', 'Tool & Maintenance Support Mechanic', 'Tool Store', 'Tool and Maintenance Support Assistant Supervisor', 'Tool and Maintenance Support Supervisor', 'Tools & Spare Part Keeper', 'Trade Analyst (Jakarta)', 'Trade Analyst (Kalimantan & Sumatera)', 'Trade Analyst (Sulawesi, Maluku & Papua)', 'Trade Analyst Coordinator (Kalimantan & Sumatera)', 'Trade Manager', 'Trade Manager (Jakarta)', 'Trade Manager (Kalimantan & Sumatera)', 'Trade Manager (Sulawesi, Maluku & Papua)', 'Trailer Coordinator', 'Trailer Maintenance Assistant Supervisor', 'Trailer Maintenance Helper', 'Trailer Maintenance Mechanic', 'Trailer Maintenance Welder', 'Trainer', 'Training & Development', 'Training And Development Crew', 'Training Supervisor', 'Transhipment Staff', 'Transportation And Vehicle Coordinator', 'Transportation Supervisor', 'Treasury Administration', 'Treasury Administration Staff', 'Treasury Administration Supervisor', 'Treasury Assistant Manager', 'Treasury Cashier', 'Treasury Core Staff', 'Treasury Core Supervisor', 'Treasury Courier', 'Treasury Manager', 'Treasury Operation Staff', 'Treasury Operation Supervisor', 'Treasury Staff', 'Treasury Supervisor', 'Troubleshooting Electrician', 'Troubleshooting Planner Staff', 'Truck Driver', 'Trucking Assistant Manager', 'Trucking Assistant Manager (JKT)', 'Trucking Assistant Manager (SBY)', 'Trucking Coordinator', 'Trucking Driver', 'Trucking Supervisor', 'Trucking Supervisor (Jakarta)', 'Tyre Crew', 'Tyre Maintenance Assistant Supervisor', 'Tyre Maintenance Helper', 'Tyre Maintenance Mechanic', 'UI / UX Designer', 'UI/UX Design Manager', 'UI/UX Designer', 'VP Investment', 'VP for Investment and Portfolio Growth', 'Vehicle Document Clearance Staff', 'Vehicle Staff', 'Vendor Management Assistant Manager', 'Vendor Management Assistant Supervisor', 'Vendor Management Ast. Manager', 'Vendor Management Staff', 'Vendor Management Supervisor', 'Vice Director', 'Vice President Director', 'Walk In & Call Centre Cust. Service Manager', 'Walk In Customer Service Executive', 'Walk In Customer Service Executive (Jakarta)', 'Walk In Customer Service Executive (Surabaya)', 'Walk In Customer Service Team Leader', 'Walk In Customer Service Team Leader (Jakarta)', 'Walk In Customer Service Team Leader (Surabaya)', 'Warehouse', 'Warehouse Assistant Manager', 'Warehouse Staff', 'Warehouse Supervisor', 'Warehouse Supervisor Jakarta Branch', 'Warehouse Worker', 'Welder Container Repair', 'Welder Heavy Equipment', 'Welder Helper', 'Welder Hull', 'Welder Repair Container', 'Welfare Staff', 'Welfare Supervisor', 'Workshop Administration Staff', 'Workshop Assistant Manager', 'Workshop Manager', 'Workshop Trainee', 'YES Purchasing Staff', 'YES Purchasing Supervisor', 'YES Representative', 'Yard Development HOD', 'Yard Development Maintenance Building', 'Yard Development Maintenance Building Staff', 'Yard Operation Administration Branch Class A East', 'Yard Operation Administration Branch Class A West', 'Yard Operation Administration Branch Class B East', 'Yard Operation Administration Branch Class B West', 'Yard Operation Administration Branch Class C East', 'Yard Operation Administration Branch Class C West', 'Yard Operation Administration Branch Class D East', 'Yard Operation Administration Jakarta Branch', 'Yard Operation Administration Staff', 'Yard Operation Administration TSP/Mirah', 'Yard Operation Assistant Manager', 'Yard Operation Assistant Supervisor', 'Yard Operation Control Coordinator', 'Yard Operation Logistics Assistant Manager', 'Yard Operation Logistics Staff', 'Yard Operation Manager', 'Yard Operation Staff Branch Class A West', 'Yard Operation Staff Branch Class B West', 'Yard Operation Staff Branch Class C East', 'Yard Operation Staff Branch Class C West', 'Yard Operation Staff Branch Class D East', 'Yard Operation Staff Branch Class D West', 'Yard Operation Supervisor', 'Yard Operation Supervisor BJTI/ Berlian & Jamrud', 'Yard Operation Supervisor Balikpapan', 'Yard Operation Supervisor Batam', 'Yard Operation Supervisor Bau-Bau', 'Yard Operation Supervisor Branch Class A East', 'Yard Operation Supervisor Branch Class A West', 'Yard Operation Supervisor Branch Class B East', 'Yard Operation Supervisor Branch Class B West', 'Yard Operation Supervisor Branch Class C East', 'Yard Operation Supervisor Branch Class C West', 'Yard Operation Supervisor Branch Class D East', 'Yard Operation Supervisor Branch Class D West', 'Yard Operation Supervisor Branch Class E East', 'Yard Operation Supervisor Hang Tuah', 'Yard Operation Supervisor Jakarta Branch', 'Yard Operation Supervisor Makassar', 'Yard Operation Supervisor Mirah & Nilam', 'Yard Operation Supervisor Sampit', 'Yard Operation Supervisor TPS', 'Yard Operation Supervisor TSP/Mirah', 'Yard Operation Supervisor Ternate', 'Yard Operations Assistant Supervisor', 'Yard Operations Head', 'Yard Operations Supervisor Gorontalo', 'Yard Operations Trainee', 'Yield Excellent Service Adm. Staff', 'Yield Excellent Service Administration Staff', 'Yield Excellent Service Assistant Manager', 'Yield Excellent Service Asst. Manager', 'Yield Excellent Service Manager', 'Yield Excellent Service Purchasing Supervisor', 'Yield Excellent Service Representative', 'Yield Excellent Service Representative Staff', 'Yield Excellent Service Representative Supervisor', 'Yield Excellent Service Supervisor']

In [45]:
import pandas as pd
from IPython.display import FileLink  # Opsional, hanya untuk Jupyter Notebook

# Fungsi untuk mengekstrak bagian department dari label dengan format "level_department_job"
def extract_department(label):
    parts = label.split('_', 2)  # Maksimal 2 pemisah
    if len(parts) == 3:
        return parts[1].strip()
    else:
        return label.strip()

# Baca file Excel input
input_file = 'matrix.xlsx'  # Pastikan file input.xlsx berada di direktori yang sama
# Asumsi: Baris pertama adalah header kolom dan kolom pertama adalah index (label baris)
df = pd.read_excel(input_file, index_col=0)

print("DataFrame Asli:")
print(df)

# Ubah label baris (index) dan kolom dengan mengekstrak department
df.index = df.index.map(extract_department)
df.columns = df.columns.map(extract_department)

# Grouping baris berdasarkan department dan jumlahkan nilainya
df_grouped_rows = df.groupby(df.index).sum()

# Grouping kolom berdasarkan department (axis=1) dan jumlahkan nilainya
df_grouped = df_grouped_rows.groupby(df_grouped_rows.columns, axis=1).sum()

print("\nDataFrame setelah grouping berdasarkan department:")
print(df_grouped)

# Normalisasi tiap baris: bagi setiap nilai dengan jumlah nilai pada baris tersebut
df_normalized = df_grouped.div(df_grouped.sum(axis=1), axis=0)

print("\nDataFrame setelah normalisasi (setiap baris berjumlah 1):")
print(df_normalized)

# Simpan DataFrame hasil normalisasi ke file Excel baru
output_file = 'aggregated_by_department_normalized.xlsx'
df_normalized.to_excel(output_file)
print("\nFile output telah disimpan sebagai:", output_file)

# Jika menggunakan Jupyter Notebook, tampilkan link download (opsional)
try:
    display(FileLink(output_file))
except Exception:
    pass


DataFrame Asli:
                                                    Director_Branches Management Department_Operation Director  \
Director_Branches Management Department_Operati...                                                  0            
Director_Business Development Department_Busine...                                                  0            
Director_Finance & Administration Department_Fi...                                                  0            
Director_Human Capital & Corporate Affairs_Huma...                                                  0            
Director_Human Capital_Corporate Affairs Director                                                   0            
...                                                                                               ...            
Worker_Yard Operations Department_Welder Repair...                                                  0            
Worker_Yield Excellent Service Department_Area ...                      

C:\Users\Alan Jonathan\AppData\Local\Temp\ipykernel_16316\2277752513.py:28: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_grouped = df_grouped_rows.groupby(df_grouped_rows.columns, axis=1).sum()



File output telah disimpan sebagai: aggregated_by_department_normalized.xlsx


d:\A_SPILL_INTERN\markov\front-end\server\aggregated_by_department_normalized.xlsx